In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
df = pd.read_spss('C:\\data\\education\\한국아동청소년행복지수조사\\kor_data_20210010.sav')
df.head(3)

,id,sex,birthy,birthm,area,size,school,grade,home1,home2,...,q512,q513,q514,q52,q53,q5411,q5412,q5421,q5422,wt
0,20001.0,여성,2003.0,4월,"충청권(대전, 충남, 충북)",도서벽지 및 읍면지역,고등학생,3학년,아니오,아니오,...,해당 없음,차이 없음,차이 없음,28.0,그대로다,NaN,NaN,NaN,NaN,0.250048
1,20002.0,여성,2003.0,8월,"충청권(대전, 충남, 충북)",도서벽지 및 읍면지역,고등학생,3학년,예,예,...,차이 없음,차이 없음,차이 없음,20.0,그대로다,NaN,NaN,NaN,NaN,0.250048
2,20003.0,여성,2003.0,9월,"충청권(대전, 충남, 충북)",도서벽지 및 읍면지역,고등학생,3학년,예,예,...,차이 없음,차이 없음,차이 없음,30.0,그대로다,NaN,NaN,NaN,NaN,0.250048


In [3]:
# for i in range(1, 8):
#     print(f"'q12{i}1','q12{i}2','q12{i}3','q12{i}4','q12{i}5', ")

In [4]:
# 기본 정보
basic_info = ['id', 'sex', 'school', 'grade']
# 친구 관계
friend = ['q05', 'q11']
about_friend = [
          'q1211', 'q1212', 'q1213', 'q1214', 'q1215', 
          'q1221', 'q1222', 'q1223', 'q1224', 'q1225', 
          'q1231', 'q1232', 'q1233', 'q1234', 'q1235', 
          'q1241', 'q1242', 'q1243', 'q1244', 'q1245', 
          'q1251', 'q1252', 'q1253', 'q1254', 'q1255', 
          'q1261', 'q1262', 'q1263', 'q1264', 'q1265', 
          'q1271', 'q1272', 'q1273', 'q1274', 'q1275'
          ]

# 교사 관계
teacher = ['q061', 'q062']

# 신뢰할 수 있는 사람
trust_person = ['q10']

# 학업 관련
study = ['q072', 'q073', 'q074']

# 사교육 관련
pri_edu = ['q09', 'q0911', 'q0912', 'q0921', 'q0922', 'q0923', 'q0924']

# 학교폭력 및 외부 폭력
school_violence = ['q33', 'q34', 'q35', 'q351', 'q352', 'q3531', 'q3532', 'q3533', 'q3534', 
                   'q3535', 'q3536', 'q3537', 'q3541', 'q3542', 'q3543', 'q3544', 'q3545', 
                   'q3546', 'q3547', 'q3551', 'q3552', 'q3553', 'q3554', 'q3555', 'q3556', 
                   'q3557', 'q36', 'q371', 'q372']

# 경제적 스트레스 및 불안
economic_stress = ['q481', 'q482', 'q483', 'q484', 'q485', 'q486', 'q487', 'q488', 'q489', 
                   'q4810', 'q4811', 'q4812', 'q4813', 'q4814', 'q4815', 'q4816', 'q4817']

# 가정 내 관계
family_rel = ['q141', 'q142', 'q143', 'q144', 'q145', 'q146', 'q147', 'q148', 'q149', 'q1410', 
              'q1411', 'q1412', 'q151', 'q152', 'q153', 'q154', 'q155', 'q161', 'q162', 'q163', 
              'q164', 'q165', 'q166', 'q167', 'q168']

# 정서적 및 사회적 삶
emotional_social = ['q221', 'q222', 'q223', 'q224', 'q225', 'q226', 'q23', 'q241', 'q242', 'q25', 
                    'q26', 'q27']

# 학교 생활과 잠, 휴식
sleep_lifestyle = ['q42', 'q43', 'q44', 'q45', 'q46', 'q391', 'q392', 'q401', 'q402', 'q41', 'q411']

# 자살, 가출 관련
suicide_runaway = ['q29', 'q301', 'q302', 'q31', 'q32']


In [5]:
def null0(df):
    """
    결측치가 있는 열을 0으로 채우는 함수
    """
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(0)
    return df


In [6]:
def encoding(df, columns, category_map): 
    df[columns] = df[columns].replace(category_map)
    return df
category4_map = {
    '전혀 그렇지 않다': 0,
    '그렇지 않다': 1,
    '그렇다': 2,
    '매우 그렇다': 3
}
category5_map = {
    '전혀 그렇지 않다': 0,
    '별로 그렇지 않다': 1,
    '보통이다' : 2,
    '조금 그렇다': 3,
    '매우 그렇다': 4
}

In [7]:
def time_min(df, hour_column, minute_column):
    # 시간과 분 열의 결측값을 0으로 채움
    df[hour_column] = df[hour_column].fillna(0)
    df[minute_column] = df[minute_column].fillna(0)
    
    # 데이터 타입을 float로 변환
    df[hour_column] = df[hour_column].astype(float)
    df[minute_column] = df[minute_column].astype(float)

    # 총 분 계산
    df['pri_edu_time'] = df[hour_column] * 60 + df[minute_column]
    return df


In [8]:
df_edu = df[study + ['q485', 'q486', 'q487', 'q488']]
df_edu = null0(df_edu)

edu_stress = ['q485','q486','q487','q488']
df_edu = encoding(df_edu, study, category4_map)
df_edu = encoding(df_edu, edu_stress, category5_map)

C:\Users\User\AppData\Local\Temp\ipykernel_23592\2448768107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns] = df[columns].replace(category_map)
C:\Users\User\AppData\Local\Temp\ipykernel_23592\2448768107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns] = df[columns].replace(category_map)


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_edu), columns=df_edu.columns)

In [10]:
def weighted_cov(X, weights):
    average = np.average(X, axis=0, weights=weights)
    X_centered = X - average
    cov_matrix = np.dot((X_centered * weights[:, None]).T, X_centered) / (weights.sum() - 1)
    return cov_matrix

In [11]:
def calculate_covariance(df, observed_vars):
    data_for_cov = df[observed_vars].apply(pd.to_numeric, errors='coerce')
    weights = df['wt'].to_numpy()
    w_cov = weighted_cov(data_for_cov.to_numpy(), weights)
    return pd.DataFrame(w_cov, index=observed_vars, columns=observed_vars)

In [12]:
def compute_vif(df, observed_vars):
    X = df[observed_vars].apply(pd.to_numeric, errors='coerce').dropna()
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

In [13]:
def fit_sem_model(model, df, w_cov_df):
    model.fit(df, cov=w_cov_df, )
    return model

In [14]:
from semopy import Model, calc_stats
model_desc = """
exam_anxiety =~ q072 + q073 + q074
academic_stress =~ q485 + q486 + q487 + q488

study_engagement =~ exam_anxiety + academic_stress 

exam_anxiety ~~ academic_stress
"""

In [15]:
df_edu['wt'] = df['wt']
observed_vars = ['q072', 'q073', 'q074', 'q485', 'q486', 'q487', 'q488']
w_cov_df = calculate_covariance(df_edu, observed_vars)

C:\Users\User\AppData\Local\Temp\ipykernel_23592\2273250911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edu['wt'] = df['wt']


In [16]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df = compute_vif(df_edu, observed_vars)

In [17]:
model = Model(model_desc)
model = fit_sem_model(model, df_edu, w_cov_df)
stats = calc_stats(model)
estimates = model.inspect()
print("적합도 지표:\n", stats.T, "\n\n")
print("모수 추정치:\n", estimates)

적합도 지표:
                       Value
DoF               11.000000
DoF Baseline      21.000000
chi2             387.765485
chi2 p-value       0.000000
chi2 Baseline  18108.106711
CFI                0.979169
GFI                0.978586
AGFI               0.959119
NFI                0.978586
TLI                0.960232
RMSEA              0.079827
AIC               33.855742
BIC              145.880640
LogLik             0.072129 


모수 추정치:
                 lval  op              rval  Estimate  Std. Err    z-value  \
0       exam_anxiety   ~  study_engagement  1.000000         -          -   
1    academic_stress   ~  study_engagement -0.041289  0.003622 -11.399127   
2               q072   ~      exam_anxiety  1.000000         -          -   
3               q073   ~      exam_anxiety  0.959061  0.015261  62.844295   
4               q074   ~      exam_anxiety  0.879349  0.015343  57.311312   
5               q485   ~   academic_stress  1.000000         -          -   
6               q486

In [18]:
factor_scores = model.predict_factors(df_edu)

df["study_engagement"] = factor_scores["study_engagement"]

print(df.head())

        id sex  birthy birthm             area         size school grade  \
0  20001.0  여성  2003.0     4월  충청권(대전, 충남, 충북)  도서벽지 및 읍면지역   고등학생   3학년   
1  20002.0  여성  2003.0     8월  충청권(대전, 충남, 충북)  도서벽지 및 읍면지역   고등학생   3학년   
2  20003.0  여성  2003.0     9월  충청권(대전, 충남, 충북)  도서벽지 및 읍면지역   고등학생   3학년   
3  20004.0  남성  2003.0     8월  충청권(대전, 충남, 충북)  도서벽지 및 읍면지역   고등학생   3학년   
4  20005.0  남성  2003.0     2월  충청권(대전, 충남, 충북)  도서벽지 및 읍면지역   고등학생   3학년   

  home1 home2  ...   q513   q514   q52   q53 q5411 q5412 q5421 q5422  \
0   아니오   아니오  ...  차이 없음  차이 없음  28.0  그대로다   NaN   NaN   NaN   NaN   
1     예     예  ...  차이 없음  차이 없음  20.0  그대로다   NaN   NaN   NaN   NaN   
2     예     예  ...  차이 없음  차이 없음  30.0  그대로다   NaN   NaN   NaN   NaN   
3     예     예  ...  차이 없음  차이 없음  30.0  그대로다   NaN   NaN   NaN   NaN   
4   아니오     예  ...  차이 없음  차이 없음  30.0  그대로다   NaN   NaN   NaN   NaN   

         wt study_engagement  
0  0.250048        -0.698716  
1  0.250048        -0.219536  
2  0.250048      